In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import pathlib

import warnings

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
# widths are the spaces for each columns in the DAT file.
widths = [
    7, #loc
    6, #type
    5, #palette
    10, #status
    3, #ind
    5, #stock_type
    24, #identifier
    5, #sheet
    5, #quantity
    6, #r
]

In [3]:
skiprows = [0,1,2,3,4,5,6,7,8,9,10]
columns_names = ["loc", "type", "palette", "status", "ind", "stock_type", "identifier", "sheet", "quantity", "r"]

In [4]:
data = {
    "steel_com_16g_4x8":{"safety":0, "dimensions":"4' X 8'", "no_item": "11100409", "description1": "STEEL 0.060\"", "description2":"[16G]", }, #STEEL_COM_16G_4X8
    "steel_com_16g_5x10":{"safety":15, "dimensions":"5' X 10'", "no_item": "11100409", "description1": "STEEL 0.060\"", "description2":"[16G]", },
    "steel_com_14g_5x10":{"safety":10, "dimensions":"5' X 10'", "no_item": "11100356", "description1": "STEEL 0.075\"", "description2":"[14G]", },
    "steel_com_11g_5x10":{"safety":54, "dimensions":"5' X 10'", "no_item": "11100638", "description1": "STEEL 0.120\"", "description2":"[11G]", },
    "steel_com_10g_5x10":{"safety":2, "dimensions":"5' X 10'", "no_item": "11100578", "description1": "STEEL 10G", "description2":"[10G]", },
    "steel_44w_0.187_5x10":{"safety":34, "dimensions":"5' X 10'", "no_item": "11100567", "description1": "STEEL 0.187\"", "description2":"[3/16G]", },
    "steel_44w_0.250_5x10":{"safety":39, "dimensions":"5' X 10'", "no_item": "11100400", "description1": "STEEL 0.250\"", "description2":"[1/4]", },
    "steel_44w_0.312_5x10":{"safety":4, "dimensions":"5' X 10'", "no_item": "11100379", "description1": "STEEL 0.312\"", "description2":"[5/16]", }, #STEEL_44W_0.312_5X10
    "steel_44w_0.375_5x10":{"safety":8, "dimensions":"5' X 10'", "no_item": "11100282", "description1": "STEEL 0.375\"", "description2":"[3/8]", },
    "steel_44w_0.500_5x10":{"safety":6, "dimensions":"5' X 10'", "no_item": "11100285", "description1": "STEEL 0.500\"", "description2":"[1/2]", },
    "steel_44w_0.625_5x10":{"safety":2, "dimensions":"5' X 10'", "no_item": "11100075", "description1": "STEEL 0.625\"", "description2":"[5/8]", "pal":"116"},
    "steel_44w_0.750_5x10":{"safety":2, "dimensions":"5' X 10'", "no_item": "11100269", "description1": "STEEL 0.750\"", "description2":"[3/4]", },
    "steel_44w_1.000_4x8":{"safety":0, "dimensions":"4' X 8'", "no_item": "11100284", "description1": "STEEL 1.000\"", "description2":"[1PO]", },
    "steel_44w_1.000_4x10":{"safety":2, "dimensions":"4' X 10'", "no_item": "11100284", "description1": "STEEL 1.000\"", "description2":"[1PO]", }, #STEEL_44W_1.000_4X10
    "steel_44w_1.000_5x10":{"safety":2, "dimensions":"5' X 10'", "no_item": "11100284", "description1": "STEEL 1.000\"", "description2":"[1PO]", },
    "steel_flr_0.187_5x10":{"safety":8, "dimensions":"5' X 10'", "no_item": "11100580", "description1": "NON SLIP STEEL 0.187\"", "description2":"[3/16]", },
    "steel_galv_11g_4x10":{"safety":3, "dimensions":"4' X 10'", "no_item": "11100525", "description1": "GALVANIZED STEEL 0.120\"", "description2":"[11G]", },
    "ss_304_16g_5x10":{"safety":5, "dimensions":"5' X 10'", "no_item": "11600005", "description1": "SS 304 0.062\"", "description2":"[16G]", },
    "ss_304_14g_5x10":{"safety":5, "dimensions":"5' X 10'", "no_item": "11600067", "description1": "SS 304 0.078\"", "description2":"[14G]", },
    "ss_304_11g_5x10":{"safety":15, "dimensions":"5' X 10'", "no_item": "11600001", "description1": "SS 304 0.125\"", "description2":"[11G]", },
    "ss_304_11g _no4_5x10":{"safety":2, "dimensions":"5' X 10'", "no_item": "171236", "description1": "SS 304 NO4 0.125\"", "description2":"[11G]", },
    "ss_304_0.250_5x10":{"safety":5, "dimensions":"5' X 10'", "no_item": "11600028", "description1": "SS 304 0.250\"", "description2":"[1/4]", },
    "ss_304_0.375_5x10":{"safety":2, "dimensions":"5' X 10'", "no_item": "11600029", "description1": "SS 304 0.375\"", "description2":"[3/8]", },
    "ss_304_0.500_5x10":{"safety":2, "dimensions":"5' X 10'", "no_item": "11600039", "description1": "SS 304 0.500\"", "description2":"[1/2]", },
    "ss_304_0.500_4x10":{"safety":0, "dimensions":"4' X 10'", "no_item": "11600039", "description1": "SS 304 0.500\"", "description2":"[1/2]", },
    "ss 304 2b 0.187 5x10":{"safety":5, "dimensions":"5' X 10'", "no_item": "11600021", "description1": "SS 304 2B 0.187\"", "description2":"[3/16]", },
    "ss_304_16g_no4_5x10":{"safety":2, "dimensions":"5' X 10'", "no_item": "171237", "description1": "SS 304 NO4 0.062\"", "description2":"[16G]", },
    "ss_304_0.187_no4_5x10":{"safety":2, "dimensions":"5' X 10'", "no_item": "212826", "description1": "SS 304 NO4 0.187\"", "description2":"[3/16]", },
    "ss_304_0.250_no4_5x10":{"safety":2, "dimensions":"5' X 10'", "no_item": "212827", "description1": "SS 304 NO4 0.250\"", "description2":"[1/4]", },
    "ss_316_16g_4X8":{"safety":0, "dimensions":"4' X 8'", "no_item": "11600152", "description1": "SS 316 16G 0.0625\"", "description2":"[1/16]", },
}

In [5]:
def Get_Dataframe(filename):
    bestand = pd.read_fwf(filename, widths=widths, colspecs='infer', header= None, skiprows = skiprows)
  
    bestand.columns = columns_names
    palettes = bestand[["identifier","palette"]]
    identifiers = palettes["identifier"].str.lower()
    palettes["identifier"] = identifiers

    palettes = palettes.set_index("identifier")
    palettes = palettes[palettes.index.notnull()]
    palettes["list"] = palettes.groupby(palettes.index)["palette"].unique()
    palettes = palettes["list"]
    palettes = palettes.to_frame(name=None)

    dict_palettes = palettes.to_dict(orient='dict')['list']

    # convert to string the arrays in the dict.
    pal_collections = {i: "-".join(j) for (i,j) in dict_palettes.items()}

    bestand['quantity'] = bestand['quantity'].astype(float)
    bestand['palette'] = bestand['palette'].astype(str)
    bestand = (
        bestand
            [["identifier","quantity"]]
            .groupby(['identifier'])['quantity'].agg(sum)
            .to_frame(name=None) # change series in dataframe.
    )

    # change the dict to a dataframe.
    cmf = pd.DataFrame.from_dict(data , orient='index', columns=['no_item','description1','description2','dimensions','safety'])
   
    # assign type to columns
    cmf = cmf.astype({"no_item": str,"description1": str,"description2":str, "dimensions":str, "safety":int})
    
    # change indexes to lower case cmf and bestand.
    cmf.index = cmf.index.str.lower()
    bestand.index = bestand.index.str.lower()
    bestand = bestand.join(cmf)

    # substract difference.
    bestand["order"] = bestand["safety"] - bestand["quantity"]
    
    # calculate surfaces.
    bestand["quantitypi2"] = bestand["order"] * 50
    
    # fill na values to zero.
    bestand[["safety","order","quantitypi2"]] = bestand[["safety","order","quantitypi2"]].fillna(0)
    
    # Rename headers
    bestand.columns = [
        "Quantity",
        "Item number",
        "Description 1",
        "Description 2",
        "Dimensions",
        "Safety stock",
        "To order",
        "Surface (ft^2)",
    ]
    
    # convert number to int.
    bestand = bestand.astype({"Quantity": int,"Safety stock": int,"To order":int, "Surface (ft^2)":int})
    
    # order headers.
    bestand = bestand[[ 'Item number', 'Description 1', 'Description 2','Dimensions', 'Safety stock', 'Quantity','To order', 'Surface (ft^2)']]

    # change negative values to zero.
    surfaces = bestand[["Surface (ft^2)"]].clip(lower=0)
    bestand["Surface (ft^2)"] = surfaces
    orders = bestand[["To order"]].clip(lower=0)
    bestand["To order"] = orders
    bestand["Palette"] = bestand.index.map(pal_collections)
    bestand = bestand[[ 'Item number', 'Palette','Description 1', 'Description 2','Dimensions', 'Safety stock', 'Quantity','To order', 'Surface (ft^2)']]
    bestand = bestand.sort_values("Palette")
    bestand = bestand.reset_index()
    return bestand

In [6]:
file = pathlib.Path("BESTAND.DAT")
if file.exists():
    print ("File: BESTAND.DAT found")
    print ("Generating speadsheet...")
    bestand = Get_Dataframe("BESTAND.DAT")
    bestand
    now = datetime.now().strftime("%Y_%m_%d(%H%M%S)")
    filename = "BESTAND_" + now + ".xlsx"
    bestand.to_excel(filename, index=False)
    print(f"File created: {filename}")
else:
    print ("File: BESTAND.DAT was not found. Please provide the file before running the macro.")

File: BESTAND.DAT found
Generating speadsheet...
File created: BESTAND_2021_07_15(142008).xlsx


In [7]:
print("press any keys to exit.")
input()

press any keys to exit.
a


'a'